In [150]:
import pandas as pd
import numpy as np
from Fin import *
from Stat import *
import warnings
warnings.filterwarnings(action='ignore')
df = pd.read_excel("Data/kospi/Alldata.xlsx")
df_main_vars = df[['RNOA', 'deltaSGA', 'deltasale', 'deltaATO', 'GNOA', 'deltaACC', 'valPEIS','consen_cut']]

# Table5_recoommand 별 통계

In [151]:
alldata, groupPEISdata = showTable5(df_main_vars)

Alldata count :  7322
consen : 0 count : 4511
consen : 1 count : 2
consen : 2 count : 254
consen : 3 count : 2544
consen : 4 count : 11


In [163]:
round(alldata,3).to_excel('Data/kospi/table5_1.xlsx')

In [164]:
round(groupPEISdata,3).to_excel('Data/kospi/table5_2.xlsx')

In [168]:
round(alldata,3)

,Mean,Std,P20,Median,P80,Min,Max
RNOA,-0.502,84.594,-2.280,0.104,1.873,-5460.711,2702.285
deltaSGA,0.001,0.435,-0.013,0.001,0.015,-25.680,25.349
deltasale,0.156,3.636,-0.074,0.036,0.171,-37.666,291.705
deltaATO,-0.011,0.444,-0.131,-0.009,0.095,-12.217,12.228
GNOA,0.113,3.654,-0.150,0.028,0.243,-108.799,184.154
deltaACC,0.715,50.099,-1.100,0.247,1.893,-1318.117,2919.699
valPEIS,0.000,1.595,-1.000,0.000,1.000,-5.000,6.000


In [169]:
round(groupPEISdata,3)

Mean     Std    P20  Median    P80       Min       Max
0 RNOA      -0.674  97.713 -2.029   0.008  1.549 -5460.711  2702.285
  deltaSGA   0.000   0.552 -0.013   0.001  0.016   -25.680    25.349
  deltasale  0.174   4.606 -0.095   0.021  0.155   -37.666   291.705
  deltaATO  -0.010   0.465 -0.138  -0.010  0.102   -12.217    12.228
  GNOA       0.122   3.918 -0.168   0.013  0.221   -64.498   184.154
  deltaACC   1.310  60.145 -1.021   0.324  2.006 -1318.117  2919.699
  PEIS       0.026   1.608 -1.000   0.000  1.000    -5.000     6.000
1 RNOA      -2.362   0.537 -2.589  -2.362 -2.134    -2.741    -1.982
  deltaSGA   0.009   0.013  0.003   0.009  0.014    -0.000     0.018
  deltasale -0.001   0.122 -0.053  -0.001  0.050    -0.088     0.085
  deltaATO   0.004   0.138 -0.054   0.004  0.063    -0.093     0.102
  GNOA      -0.118   0.002 -0.118  -0.118 -0.117    -0.119    -0.116
  deltaACC   1.375   4.056 -0.346   1.375  3.096    -1.493     4.243
  PEIS       0.000   2.828 -1.200   0.000  1.200    -2.000     2.000
2 RNOA      -0.699  19.501 -2.214  -0.021  1.106  -177.316   184.293
  deltaSGA   0.009   0.064 -0.011   0.002  0.021    -0.182     0.626
  deltasale  0.001   0.207 -0.122  -0.005  0.098    -0.908     0.954
  deltaATO  -0.080   0.309 -0.174  -0.033  0.037    -2.721     1.142
  GNOA       0.079   1.712 -0.092   0.049  0.246   -16.733    12.850
  deltaACC   0.679  15.045 -1.474   0.090  1.604   -42.167   211.975
  PEIS      -0.291   1.522 -1.000   0.000  1.000    -4.000     5.000
3 RNOA      -0.180  60.255 -2.880   0.277  2.510 -1458.182  1372.175
  deltaSGA   0.000   0.063 -0.011   0.001  0.013    -1.780     0.822
  deltasale  0.140   0.658 -0.030   0.064  0.200    -0.982    15.200
  deltaATO  -0.006   0.418 -0.111  -0.007  0.088    -7.100     6.947
  GNOA       0.102   3.306 -0.127   0.052  0.266  -108.799    94.601
  deltaACC  -0.339  28.036 -1.209   0.163  1.718  -850.355   388.584
  PEIS      -0.018   1.573 -1.000   0.000  1.000    -5.000     6.000
4 RNOA       0.426   2.729 -0.724   0.497  1.161    -4.722     6.482
  deltaSGA   0.037   0.135 -0.008   0.002  0.023    -0.063     0.439
  deltasale  0.355   0.750 -0.167   0.096  0.823    -0.499     2.121
  deltaATO  -0.143   0.386 -0.483  -0.111  0.098    -0.852     0.560
  GNOA      -0.274   1.096 -0.384  -0.123  0.149    -3.281     1.224
  deltaACC   0.931   2.027 -0.192   0.575  2.101    -3.082     4.827
  PEIS       0.091   2.023 -2.000   0.000  1.000    -3.000     4.000

#

#

#

#

#

# Table6

In [154]:
Alldata = pd.read_excel("Data/kospi/Alldata.xlsx")
UseOnlyPort = Alldata[['회계년','Name','시가총액','consen_cut','CompuConsen','valPEIS','deltaprice','deltaearn']]

# 컨센별ALL & 컨센안에서 PEIS별

In [155]:
ConsenPorts = MakePortbyCriteria(UseOnlyPort, 'consen_cut')
for i in ConsenPorts:
    i.ConsenPEIS()

[0, 1, 2, 3, 4]


In [156]:
AllDatabyConsen = MakeTable6form(ConsenPorts)

In [157]:
table = []
for df in ConsenPorts:
    table.append(MakeTable6form(df.PEIS))
PEISConsentable = pd.concat(table)

In [174]:
Table6 = pd.concat([AllDatabyConsen, PEISConsentable])
Table6.index = [ ['All']*5+['No consen']*5+['Sell']*5+['Hold']*5+['Buy']*5+['S_Buy']*5, [5,4,3,2,1]*6   ]

In [175]:
Table6.to_excel('Data/kospi/table6.xlsx')
Table6.fillna(0)

Earning increases  Earning decrease  Percentage of increse
All       5             2237.0            2274.0                   0.50
          4                0.0               2.0                   0.00
          3              119.0             135.0                   0.47
          2             1320.0            1224.0                   0.52
          1                5.0               6.0                   0.45
No consen 5              402.0             365.0                   0.52
          4              446.0             467.0                   0.49
          3                0.0               0.0                   0.00
          2              562.0             558.0                   0.50
          1              827.0             884.0                   0.48
Sell      5                0.0               1.0                   0.00
          4                0.0               0.0                   0.00
          3                0.0               0.0                   0.00
          2                0.0               0.0                   0.00
          1                0.0               1.0                   0.00
Hold      5               15.0              30.0                   0.33
          4                0.0               0.0                   0.00
          3               26.0              42.0                   0.38
          2               40.0              29.0                   0.58
          1               38.0              34.0                   0.53
Buy       5              211.0             200.0                   0.51
          4              310.0             256.0                   0.55
          3                0.0               0.0                   0.00
          2              336.0             330.0                   0.50
          1              463.0             438.0                   0.51
S_Buy     5                0.0               1.0                   0.00
          4                2.0               1.0                   0.67
          3                2.0               0.0                   1.00
          2                0.0               0.0                   0.00
          1                1.0               4.0                   0.20

#

#

#

#

#
#

# Table 1

In [171]:
# 데이터 준비시키기
Table1_Series = UseOnlyPort.groupby(['회계년', 'consen_cut']).count()['Name']
Table1 = Table1_Series.unstack(1)

# 컬럼합 보여주기
colsum = Table1.sum().tolist()
newrow = len(Table1)
Table1.loc[newrow] = colsum
newrow = len(Table1)
Table1.loc[newrow] = Table = round(Table1.loc[newrow-1]/Table1.loc[newrow-1].sum(),2).tolist()
Table1.index = [ i for i in range(2010,2020)] + ['Total'] + ['Percentage']

# 로우합보여주기
rowsum = Table1.sum(axis=1)
Table1 = pd.concat([Table1, rowsum], axis = 1)
Table1.columns = ['No consen', 'Sell', 'Hold', 'Buy', 'S_Buy','Total']
# 마지막으로 보여주기
Table1.fillna(0,inplace = True)
Table1.to_excel('Data/kospi/table1.xlsx')
Table1

,No consen,Sell,Hold,Buy,S_Buy,Total
2010,441.00,0.0,23.00,241.00,3.0,708.0
2011,478.00,0.0,19.00,227.00,3.0,727.0
2012,467.00,0.0,24.00,227.00,1.0,719.0
2013,453.00,1.0,23.00,240.00,0.0,717.0
2014,436.00,0.0,24.00,264.00,2.0,726.0
2015,413.00,0.0,27.00,289.00,1.0,730.0
2016,436.00,0.0,35.00,262.00,1.0,734.0
2017,439.00,1.0,28.00,274.00,0.0,742.0
2018,467.00,0.0,30.00,261.00,0.0,758.0
2019,481.00,0.0,21.00,259.00,0.0,761.0
